# 地图数据来源

#### 数据来源

https://news.qq.com/zt2020/page/feiyan.htm

数据源：https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5

In [3]:
import requests
import json#获取数据是json格式
import pandas as pd#数据整理使用
from copy import deepcopy

#爬虫函数
def getData():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
    }
    #定义
    r = requests.get(url,headers)
    #res获取最终结果；从requests中获取值，有text、json两种方式，此页面在传输时以html格式传输，不用r.json转码，用r.text
    res = json.loads(r.text)
    #再进行第二层转换，数据中我们取得data后面的，前面的对我们没用；
    data_res=json.loads(res['data'])
    return data_res
#调用
data_dict = getData()

In [4]:
#查看有哪些健
data_dict.keys()

dict_keys(['lastUpdateTime', 'chinaTotal', 'chinaAdd', 'isShowAdd', 'showAddSwitch', 'areaTree', 'chinaDayList', 'chinaDayAddList', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList'])

In [5]:
chinaTotal = deepcopy(data_dict['chinaTotal'])
chinaAdd = deepcopy(data_dict['chinaAdd'])
country = deepcopy(data_dict['areaTree'])
province = deepcopy(data_dict['areaTree'])
province_city = deepcopy(data_dict['areaTree'])

In [6]:
country[0].pop('children')
datasets = list()
for c in country:
    info = dict()
    info['country_name'] = c['name']
    info['today_confirm'] = c['today']['confirm']
    info['total_confirm'] = c['total']['confirm']
    info['suspect'] = c['total']['suspect']
    info['dead'] = c['total']['dead']
    info['deadRate'] = c['total']['deadRate']
    info['heal'] = c['total']['heal']
    info['healRate'] = c['total']['healRate']
    info['create_dt'] = data_dict['lastUpdateTime']
    datasets.append(info)
country = pd.DataFrame(datasets)
country.create_dt = pd.to_datetime(country.create_dt)

In [7]:
country

,country_name,today_confirm,total_confirm,suspect,dead,deadRate,heal,healRate,create_dt
0,中国,42,81458,106,3261,4.00,71881,88.24,2020-03-22 06:32:34


#### 参数解析：

lastUpdateTime：最后的更新时间

chinaTotal：累计
- confirm:累计确诊
- heal：累计治疗
- dead: 累计死亡
- nowCofirm: 现有确诊
- suspect: 现有疑似
- nowsevere: 现有重症

chinaAdd： 每日新增数据
- confirm:较上日累计确诊
- heal：较上日累计治疗
- dead: 较上日累计死亡
- nowCofirm: 较上日现有确诊
- suspect: 较上日现有疑似
- nowsevere: 较上日现有重症

areaTree： 树桩形状的全球的精准数据(国家、省份、城市都有所记录，数据很大)


#### 使用库

pip install pyecharts

pip install echarts-china-provinces-pypkg

pyecharts官网文档: https://pyecharts.org

In [8]:
#拿到数据，进行整理,选择我们想要拿到的数据，建立列表
province_list = data_dict['areaTree'][0]['children']
proList = list()
#每一省份遍历
for province in province_list:
    #再次筛选自己想要的数据
    pro_dict = dict()
    pro_dict['name'] = province['name']
    pro_dict['confirm'] = province['total']['confirm']
    pro_dict['heal'] = province['total']['heal']
    pro_dict['dead'] = province['total']['dead']  
    proList.append(pro_dict)
proList

[{'name': '湖北', 'confirm': 67800, 'heal': 58942, 'dead': 3139},
 {'name': '香港', 'confirm': 273, 'heal': 98, 'dead': 4},
 {'name': '台湾', 'confirm': 153, 'heal': 28, 'dead': 2},
 {'name': '北京', 'confirm': 504, 'heal': 396, 'dead': 8},
 {'name': '广东', 'confirm': 1400, 'heal': 1325, 'dead': 8},
 {'name': '上海', 'confirm': 380, 'heal': 327, 'dead': 3},
 {'name': '甘肃', 'confirm': 134, 'heal': 113, 'dead': 2},
 {'name': '浙江', 'confirm': 1236, 'heal': 1219, 'dead': 1},
 {'name': '黑龙江', 'confirm': 484, 'heal': 463, 'dead': 13},
 {'name': '澳门', 'confirm': 18, 'heal': 10, 'dead': 0},
 {'name': '山东', 'confirm': 764, 'heal': 749, 'dead': 7},
 {'name': '福建', 'confirm': 303, 'heal': 295, 'dead': 1},
 {'name': '陕西', 'confirm': 248, 'heal': 239, 'dead': 3},
 {'name': '四川', 'confirm': 542, 'heal': 536, 'dead': 3},
 {'name': '河北', 'confirm': 318, 'heal': 310, 'dead': 6},
 {'name': '云南', 'confirm': 176, 'heal': 172, 'dead': 2},
 {'name': '辽宁', 'confirm': 126, 'heal': 122, 'dead': 2},
 {'name': '广西', 'confi

In [9]:
#转变成dataframe，一个列表包含字典，一个字典表示每一列的列名称 
province_df = pd.DataFrame(proList)
province_df

,name,confirm,heal,dead
0,湖北,67800,58942,3139
1,香港,273,98,4
2,台湾,153,28,2
3,北京,504,396,8
4,广东,1400,1325,8
5,上海,380,327,3
6,甘肃,134,113,2
7,浙江,1236,1219,1
8,黑龙江,484,463,13
9,澳门,18,10,0


In [10]:
#现确诊人数： = 累计人数-累计治愈人数-累计死亡人数   #添加到dataframe中
province_df['nowConfirm'] = province_df.confirm - province_df.heal - province_df.dead

In [11]:
province_df

,name,confirm,heal,dead,nowConfirm
0,湖北,67800,58942,3139,5719
1,香港,273,98,4,171
2,台湾,153,28,2,123
3,北京,504,396,8,100
4,广东,1400,1325,8,67
5,上海,380,327,3,50
6,甘肃,134,113,2,19
7,浙江,1236,1219,1,16
8,黑龙江,484,463,13,8
9,澳门,18,10,0,8


# 绘制地图

In [12]:
from pyecharts.charts import Map
#共用插件，控制图片大小、标题颜色
from pyecharts import options as opts

In [14]:

#添加参数series_name: str,字符串类型；data_pair: Sequence,序列，需要生成器zip合成的类型，两个列表组合在一起的zip([key],[value])
#取值前做for循环,变成列表类型
province_name = [str(x) for x in province_df.name]
province_confirm = [int(x) for x in province_df.confirm]
province_nowConfirm = [int(x) for x in province_df.nowConfirm]
province_heal = [int(x) for x in province_df.heal]
province_dead = [int(x) for x in province_df.dead]


#生成器，二维列表类型#使用dataframe取出来的series列转换的list，每一个值的数据类型依然是int64，不是python中的int；莫把引擎对series不支持，你取得值必须int()强制转换
gen1 = [list(z) for z in zip(province_name,province_df.confirm)]
gen2 = [list(z) for z in zip(province_name,province_df.confirm)]
gen3 = [list(z) for z in zip(province_name,province_df.confirm)]
#第三个参数：maptype: str = 'china',默认中国地图；

#颜色指定
pieces = [
    {'max':1,'color':'#F0F0F0'},
    {'min':1,'max':9,'color':'#FFE0E0'},
    {'min':10,'max':99,'color':'#FEC0C0'},
    {'min':100,'max':499,'color':'#FD9090'},
    {'min':500,'max':999,'color':'#FC6060'},
    {'min':1000,'max':9999,'color':'#FB3030'},
    {'min':10000,'color':'#DD0000)'},
]
'''
#对地图实例化设置，地图颜色设置
c = Map(opts.InitOpts(width='900px',height='500px'))
c.add('全国累计确诊',gen1,'china')
c.add('全国累计治愈',gen2,'china',is_selected=False)#is_selected=False默认是不被选中状态
c.add('全国累计死亡',gen3,'china',is_selected=False)
#对地图的参数全局配置做调节
c.set_global_opts(title_opts=opts.TitleOpts(title='中国加油！武汉加油'),visualmap_opts=opts.VisualMapOpts(is_piecewise=True,pieces=pieces))
'''
c=(
    Map(opts.InitOpts(width='900px',height='500px'))
    .add('全国累计确诊',gen1,'china')
    .add('全国累计治愈',gen2,'china',is_selected=False)
    .add('全国累计死亡',gen3,'china',is_selected=False)
    .set_global_opts(title_opts=opts.TitleOpts(title='中国加油！武汉加油'),visualmap_opts=opts.VisualMapOpts(is_piecewise=True,pieces=pieces))
)

'''
#当存数据保存下来
c.render_embed()
from IPython.display import HTML
#把html做渲染；jupyter可能不认为，HTML帮你做渲染；这里可能不太友好，出来的是空白
#HTML(c.render_embed())
'''

c.render_notebook()

In [82]:
c=(
    Map(opts.InitOpts(width='900px',height='500px'))
    .add('全国累计确诊',[],'河北')
)
c.render_notebook()

In [15]:
#查看有哪些健
pr = deepcopy(data_dict['areaTree'])
pr

[{'name': '中国',
  'today': {'confirm': 42, 'isUpdated': True},
  'total': {'nowConfirm': 6316,
   'confirm': 81458,
   'suspect': 106,
   'dead': 3261,
   'deadRate': '4.00',
   'showRate': False,
   'heal': 71881,
   'healRate': '88.24',
   'showHeal': True},
  'children': [{'name': '湖北',
    'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False, 'tip': ''},
    'total': {'nowConfirm': 5719,
     'confirm': 67800,
     'suspect': 0,
     'dead': 3139,
     'deadRate': '4.63',
     'showRate': False,
     'heal': 58942,
     'healRate': '86.94',
     'showHeal': True},
    'children': [{'name': '武汉',
      'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False},
      'total': {'nowConfirm': 5610,
       'confirm': 50005,
       'suspect': 0,
       'dead': 2504,
       'deadRate': '5.01',
       'showRate': False,
       'heal': 41891,
       'healRate': '83.77',
       'showHeal': True}},
     {'name': '孝感',
      'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': Fa

In [16]:
pr = pr[0].pop('children')
pr

[{'name': '湖北',
  'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False, 'tip': ''},
  'total': {'nowConfirm': 5719,
   'confirm': 67800,
   'suspect': 0,
   'dead': 3139,
   'deadRate': '4.63',
   'showRate': False,
   'heal': 58942,
   'healRate': '86.94',
   'showHeal': True},
  'children': [{'name': '武汉',
    'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False},
    'total': {'nowConfirm': 5610,
     'confirm': 50005,
     'suspect': 0,
     'dead': 2504,
     'deadRate': '5.01',
     'showRate': False,
     'heal': 41891,
     'healRate': '83.77',
     'showHeal': True}},
   {'name': '孝感',
    'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False},
    'total': {'nowConfirm': 30,
     'confirm': 3518,
     'suspect': 0,
     'dead': 128,
     'deadRate': '3.64',
     'showRate': False,
     'heal': 3360,
     'healRate': '95.51',
     'showHeal': True}},
   {'name': '鄂州',
    'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False},
    'total': {'nowCo

In [17]:
#例子：  生成器只允许拿1次；
z = zip(list('abcde'),range(5))
list(z)
dict(z)

{}

In [19]:
#例子
from IPython.display import HTML
HTML('<h1>1234<h1/>')